In [19]:
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Convolution2D , Cropping2D, SeparableConv2D
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Cropping2D, Lambda, concatenate, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os
import json
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
%matplotlib inline

In [20]:
def test_model():
    img_in = Input(shape=(120, 160,3), name='img_in')  
    x =  Cropping2D(cropping=((50, 0), (0, 0)))(img_in)
    x = Convolution2D(32, (3, 3), strides=(2, 2), padding='same')(x)
    
    filte1 = 16
    filte2 = 24
    filte3 = 32
    filte4 = 40
    filte5 = 48
    
    ############################################# 1
    x = Convolution2D(filte1, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
        
    x = SeparableConv2D(filte1, (3, 3), strides=(2, 2), depth_multiplier=1, padding='same')(x)     
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(filte1, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    ############################################## 2
    x = Convolution2D(filte2 * 6, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
        
    x = SeparableConv2D(filte2, (3, 3), strides=(2, 2), depth_multiplier=1, padding='same')(x)     
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(filte2, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    ############################################ 3
    x = Convolution2D(filte3 * 6, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
        
    x = SeparableConv2D(filte3, (3, 3), strides=(2, 2), depth_multiplier=1, padding='same')(x)     
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(filte3, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    ############################################ 4
    x = Convolution2D(filte4 * 6, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
        
    x = SeparableConv2D(filte4, (3, 3), strides=(2, 2), depth_multiplier=1, padding='same')(x)     
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(filte4, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    ############################################ 5
    x = Convolution2D(filte5, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
        
    x = SeparableConv2D(filte5, (3, 3), strides=(2, 2), depth_multiplier=1, padding='same')(x)     
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(filte5, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    ############################################ end
    
   
    x = Convolution2D(1280, (1, 1), strides=(1, 1), padding='same')(x)
    x = GlobalAveragePooling2D()(x)
   
    
    x = Dense(100, activation='linear')(x)
    x = Dropout(.1)(x)
    x = Dense(50, activation='linear')(x)
    x = Dropout(.1)(x)  
      
    angle_out = Dense(1, activation='tanh', name='angle_out')(x)  
    throttle_out = Dense(1, activation='sigmoid', name='throttle_out')(x)  
    
    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])
    model.compile(optimizer='adam',
                  loss={'angle_out': 'mean_squared_error',
                        'throttle_out': 'mean_squared_error'},
                  loss_weights={'angle_out': 0.5, 'throttle_out': 0.5})
    return model

In [22]:
real_df = pd.read_csv('real_df.csv')

In [23]:
# real_df.head()

In [24]:
def select_rgb_white_yellow(image): 
    # white color mask
    lower = np.uint8([0, 0, 0])
    upper = np.uint8([100, 100, 100])
    white_mask = cv2.inRange(image, lower, upper)
    # yellow color mask
    lower = np.uint8([0, 0, 0])
    upper = np.uint8([50, 50, 50])
    yellow_mask = cv2.inRange(image, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked = cv2.bitwise_and(image, image, mask = mask)
    return masked

In [25]:
def get_transfer_data_simu(img):     
    img = Image.open(img)
    for i in range(img.size[0]):    
        for j in range(img.size[1]):
            if j > 50:
                r,g,b = img.getpixel((i,j))
                if(r > 190 and g > 200 and b > 220):
                     img.putpixel((i,j), (34,37,23))
                elif(r > 200 and g > 190 and b < 180 and b > 120):  
                     img.putpixel((i,j), (52,33,30))
   
    img = select_rgb_white_yellow(np.array(img))    
    return img

In [26]:
def get_transfer_data_simu_canny(img):     
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img = cv2.Canny(img,50,300)
    img = np.stack((img,img,img), axis=2)   
    return img

In [27]:
def get_transfer_data_real(img):     
    img = Image.open(img)    
    img = select_rgb_white_yellow(np.array(img))               
    return img

In [28]:
def get_transfer_data_real_canny(img):
    loc = img
    try:
        img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        img = cv2.Canny(img,100,400)
        img = np.stack((img,img,img), axis=2)
        return img
    except:
        print('error: '+ loc)
        img = 'error'
        return img

In [29]:
start_time = time.time()

real_df["img"] = real_df["cam/image_array"].apply(lambda x: get_transfer_data_real_canny(x))
print(real_df.shape)
real_df = real_df[real_df['img'] != 'error']
real_df.head()
print(real_df.shape)

elapsed_time = time.time() - start_time
peroid = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print(peroid)

(6293, 7)
(6293, 7)
00:00:30


/opt/conda/lib/python3.6/site-packages/pandas/core/ops.py:1122: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x, y, op)


In [30]:
def get_img_list(df):
    img_list = []
    for i in df["img"].values:
        img_list.append(i)  
    return np.array(img_list)     

In [31]:
def train_model(df, model, name):
    early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=.0005,
                                   patience=5,
                                   verbose=2,
                                   mode='auto')
    save_best = ModelCheckpoint(name+'_model',
                                    monitor='val_loss',
                                    verbose=2,
                                    save_best_only=True,
                                    mode='min')
    model.fit(x=get_img_list(df),
                 y=[df["user/angle"].values,df["user/throttle"].values],
                 batch_size=32,
                 epochs=100,
                 verbose=1,
                 callbacks=[early_stop,save_best],             
                 validation_split=0.1
                )

In [32]:
def build_speed_models(df, range_list, name):
    for throttle in range_list:
        new_model = test_model()
        df["user/throttle"] = throttle
        train_model(df,new_model, name + str(throttle))
    

In [33]:
# real_df['user/angle'][real_df['user/angle'] > 0.5] = 0.7
# real_df['user/angle'][real_df['user/angle'] < -0.5] = -0.7

In [34]:
# build_speed_models(real_df,[0.8], 'real_angle2_throttle_')

In [35]:
#############transfer

In [17]:
def build_speed_models_transfer(df, model, range_list, name):
    for throttle in range_list:       
        df["user/throttle"] = throttle
        train_model(df,model, name + str(throttle))

In [36]:
build_speed_models_transfer(real_df,  load_model('old_mix_data/mix_model_model'), [0.9,1.0], 'transfer_20181025_light_')

Train on 5663 samples, validate on 630 samples
Epoch 1/100
5663/5663 [==============================] - 50s 9ms/step - loss: 0.0459 - angle_out_loss: 0.0843 - throttle_out_loss: 0.0074 - val_loss: 0.0347 - val_angle_out_loss: 0.0694 - val_throttle_out_loss: 5.7084e-05

Epoch 00001: val_loss improved from inf to 0.03473, saving model to transfer_20181025_light_0.9_model
Epoch 2/100
5663/5663 [==============================] - 48s 8ms/step - loss: 0.0326 - angle_out_loss: 0.0644 - throttle_out_loss: 7.5454e-04 - val_loss: 0.0286 - val_angle_out_loss: 0.0568 - val_throttle_out_loss: 3.1661e-04

Epoch 00002: val_loss improved from 0.03473 to 0.02858, saving model to transfer_20181025_light_0.9_model
Epoch 3/100
5663/5663 [==============================] - 49s 9ms/step - loss: 0.0294 - angle_out_loss: 0.0584 - throttle_out_loss: 4.4148e-04 - val_loss: 0.0322 - val_angle_out_loss: 0.0640 - val_throttle_out_loss: 4.0356e-04

Epoch 00003: val_loss did not improve
Epoch 4/100
5663/5663 [=======